# Empty Response Test

In [1]:
import pytest
from unittest.mock import AsyncMock, patch, MagicMock
from uuid import uuid4

from dotenv import load_dotenv
import django

In [2]:
load_dotenv("../tests/.env.test")
django.setup()

2025-05-08 12:35:29,440 DEBUG selector_events: Using selector: KqueueSelector
2025-05-08 12:35:30,467 DEBUG environ: get 'ENVIRONMENT' casted as '<class 'str'>' with default '<NoValue>'
2025-05-08 12:35:30,468 DEBUG environ: get 'COLLECTION_ENDPOINT' casted as '<class 'str'>' with default '<NoValue>'
2025-05-08 12:35:30,468 INFO settings: the parsed url is ParseResult(scheme='http', netloc='admin:Opensearch2024^@localhost:9200', path='', params='', query='', fragment='')
2025-05-08 12:35:30,469 DEBUG environ: get 'ENABLE_METADATA_EXTRACTION' casted as '<class 'bool'>' with default 'True'
2025-05-08 12:35:30,469 DEBUG environ: get 'DATAHUB_REDBOX_URL' casted as '<class 'str'>' with default ''
2025-05-08 12:35:30,470 DEBUG environ: get 'DATAHUB_REDBOX_SECRET_KEY' casted as '<class 'str'>' with default ''
2025-05-08 12:35:30,470 DEBUG environ: get 'DATAHUB_REDBOX_ACCESS_KEY_ID' casted as '<class 'str'>' with default ''
2025-05-08 12:35:30,496 DEBUG environ: get 'LLM_MAX_TOKENS' casted as 

In [3]:
from redbox_app.redbox_core.consumers import ChatConsumer

2025-05-08 12:35:31,441 INFO consumers: WEBSOCKET_SCHEME is: ws
2025-05-08 12:35:31,441 INFO settings: Testing OpenSearch is definitely being used
2025-05-08 12:35:31,442 DEBUG connectionpool: Starting new HTTP connection (1): localhost:9200
2025-05-08 12:35:31,446 DEBUG connectionpool: http://localhost:9200 "HEAD /_alias/redbox-data-test-chunk-current HTTP/11" 200 0
2025-05-08 12:35:31,447 INFO base: HEAD http://localhost:9200/_alias/redbox-data-test-chunk-current [status:200 request:0.005s]
2025-05-08 12:35:31,447 DEBUG base: > None
2025-05-08 12:35:31,447 DEBUG base: < 
2025-05-08 12:35:31,451 DEBUG connectionpool: http://localhost:9200 "HEAD /redbox-data-test-chat-mesage-log HTTP/11" 200 0
2025-05-08 12:35:31,452 INFO base: HEAD http://localhost:9200/redbox-data-test-chat-mesage-log [status:200 request:0.005s]
2025-05-08 12:35:31,452 DEBUG base: > None
2025-05-08 12:35:31,452 DEBUG base: < 
2025-05-08 12:35:31,452 INFO settings: Testing OpenSearch is definitely being used
2025-05-0

In [4]:
@pytest.mark.asyncio
async def test_llm_conversation_with_empty_response():
    consumer = ChatConsumer(scope={"user", MagicMock(id=uuid4())})

    # Mock Chat and Files
    mock_session = MagicMock()
    mock_user = MagicMock(id=uuid4())
    mock_file = MagicMock(unique_name="file1")
    selected_files = [mock_file]
    permitted_files = [mock_file]

    # Mock Chat Messages
    chat_message1 = MagicMock(role="user", text="Hello")
    chat_message2 = MagicMock(role="ai", text="")  # Blank
    # replaced django_app.redbox_app.redbox_core.models.ChatMessage by redbox_app.redbox_core.models.ChatMessage
    with (
        patch("redbox_app.redbox_core.models.ChatMessage") as mock_filter,
        patch.object(consumer, "get_ai_settings", new_callable=AsyncMock) as mock_get_ai_settings,
        patch.object(consumer.redbox, "run", new_callable=AsyncMock) as mock_redbox_run,
        patch.object(consumer, "send_to_client", new_callable=AsyncMock) as mock_send_to_client,
        patch.object(consumer, "save_ai_message", new_callable=AsyncMock),
    ):
        mock_filter.return_value.order_by.return_value.__aiter__.return_value = [chat_message1, chat_message2]
        mock_get_ai_settings.return_value = {}
        mock_redbox_run.return_value = {}

        with pytest.raises(ValueError, match="Null LLM Response Received"):
            await consumer.llm_conversation(selected_files, mock_session, mock_user, "Test Title", permitted_files)
        mock_send_to_client.assert_called()

In [5]:
await test_llm_conversation_with_empty_response()

ValidationError: ['“[]” is not a valid UUID.']